<a href="https://colab.research.google.com/github/JayroMartinez/game_of_life_sim/blob/main/game_of_life_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/JayroMartinez/game_of_life_sim.git -q
!pip install -q -r game_of_life_sim/requirements.txt
import sys; sys.path.append("/content/game_of_life_sim/src")
%cd /content/game_of_life_sim

In [ ]:
!python run_sim.py --size 60 --prob 0.25

In [ ]:
!python run_sim.py --runs 10000 --size 60 --prob 0.25

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

!mkdir -p ~/.ssh
!cp /content/drive/MyDrive/keys/id_ed25519_life ~/.ssh/
!chmod 600 ~/.ssh/id_ed25519_life
!ssh-keyscan github.com >> ~/.ssh/known_hosts

!git add data/results.csv
!git commit -m "Add new results"
!git push git@github.com:<tu-usuario>/game_of_life_sim.git HEAD:main